In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pandas_ml_utils import pd, np, SkModel, FeaturesAndLabels, stratified_random_splitter
from pandas_ml_utils_test.config import DF_NOTES
import matplotlib.pyplot as plt

The following data set contains variables to determine whether a Note is valid or not

In [2]:
DF_NOTES.tail()

,variance,skewness,kurtosis,entropy,authentic
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1
1371,-2.54190,-0.65804,2.6842,1.19520,1


Now lets estimate which features might be useful to predict the the label whether a note is authentic (1) or not (0).
But before ew do that we add some redundancy and some random data. The feature selection should obviously be able to
get rid of such useless data. Since we do not know if the data is sorted in some way we use a
`stratified_random_splitter` to make sure that we have equally many instance of each class in the training and test set.

In [3]:
# make experiment reproducable
np.random.seed(42)

res = DF_NOTES.model.feature_selection(
     features_and_labels=FeaturesAndLabels(
        features=["variance", "skewness", "kurtosis", "entropy",
                  lambda df: (df["skewness"] * 0.5).rename("skewness red"),       # add correlated data
                  lambda df: pd.Series(np.random.random(len(df)), name='noise'),  # add random noise
                  ],
        labels=["authentic"],
        label_type=int
    ),
    training_data_splitter=stratified_random_splitter(0.3),
    correlated_features_th=0.8                                                    # keep kurtosis
)

res

drop redundant features: {'skewness': 1.0}


Data was not in RNN shape
Data was not in RNN shape
Data was not in RNN shape
Data was not in RNN shape
Data was not in RNN shape
Data was not in RNN shape
Data was not in RNN shape


As we can see, the feature selection process got rid of the two unnecessary variables. It is also worth to note that
the remaining features are equally important in the training and test data set. It is a pretty useful heuristic to chek
if the features are equally informative on out of sample data as well. Because if this is not the case we will only
over fit in the best case.

Now we can just re-use the best estimated model (which is a RandomForrest classifier / regressor):
``` python
skmodel = res.model.sk_model.best_estimator_.estimator_
```

Or we use a different type of estimator, i.e. we might want to use a Multi Layer Perceptron. In order to get a very
rough idea of the network size we could just use the number of nodes from the regression trees.

```python
from sklearn.neural_network import MLPClassifier
network_size = res.training_summary.nr_of_nodes
nr_layers = 2

hidden = [int(network_size ** (1 / float(nr_layers)))] * nr_layers
print("hidden size", hidden)

skmodel = MLPClassifier(hidden_layer_sizes=hidden, activation='tanh')
```

For a quick experiment we would be good to go and just invoke `df.model.fit(SkModel(skmodel, FeaturesAndLabels(...))`.
But we want to save the model and load it painless in any context we like to i.e. in a flask application. This means
we do not care about any modules which might be imported. And we do not need to know if the raw data needs to be
pre-processed in some form. Therefore we re-assign the model in a model context.

In [5]:
with DF_NOTES.model("notes-$V.model") as m:  # note the $V makro allows to save multiple versions of the model
    from pandas_ml_utils import SkModel, FeaturesAndLabels, ClassificationSummary, stratified_random_splitter
    from sklearn.neural_network import MLPClassifier
    
    network_size = res.training_summary.nr_of_nodes
    nr_layers = 2

    hidden = [int(network_size ** (1 / float(nr_layers)))] * nr_layers
    print("hidden size", hidden)

    fit = m.fit(
        SkModel(
            MLPClassifier(hidden_layer_sizes=hidden, activation='tanh'),
            FeaturesAndLabels(
                features=["variance", "skewness", "kurtosis"],
                labels=["authentic"]
            ),
            summary_provider=ClassificationSummary
        ),
        training_data_splitter=stratified_random_splitter(0.25)
    )

fit

hidden size [24, 24]
Data was not in RNN shape
Data was not in RNN shape
Data was not in RNN shape
saved model to: /home/kic/sources/private/projects/pandas-quant/pandas-ml-utils/examples/notes-202010-15-59240.model


Here we are with a nicely fitted model :tada:

We should also note that our model has been saved. So we can create any new script, app, etc. and just load it:
```python
from pandas_ml_utils import Model

df = load_a_notes_data_frame()
model = Model.load('notes-202010-15-56730.model')
prediction = df.model.predict(model)
```